In [1]:
!pip install --index-url https://pypi.org/simple llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 25.7 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp310-cp310-linux_x86_64.whl size=4601128 sha256=dbf441f9480c1594f00199998abc241d3d6ed16ab96102d820f75d06b8e6b3ec
  Stored in directory: /root/.cache/pip/wheels/5c/8f/58/a39eb13258f3bbf64bb36ed76d31979579a6f175be38de06b7
Successfully built llama-cpp-python


In [2]:
!wget -O /kaggle/working/llama-2-7b-chat.gguf.q4_K_M.bin "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf"

--2025-02-15 01:57:01--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.59, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1739588221&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU4ODIyMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg3Mzg1M

In [3]:
from llama_cpp import Llama

model_path = "/kaggle/working/llama-2-7b-chat.gguf.q4_K_M.bin"
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /kaggle/working/llama-2-7b-chat.gguf.q4_K_M.bin (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count

In [4]:
import pandas as pd
import re
import random
import textwrap

# Charger les données
df = pd.read_csv('/kaggle/input/twitterr/tweets.csv')

# Nettoyer hashtags et mentions
df['Hashtags'] = df['Hashtags'].dropna().str.lower()
df['Mentions'] = df['Mentions'].dropna().str.lower()

# Extraire les hashtags les plus utilisés
hashtags_list = df['Hashtags'].str.split(r'[ ,]').explode().dropna()
top_hashtags = hashtags_list.value_counts().head(10)

# Extraire les mentions les plus utilisées
top_users = df['Mentions'].str.split(r'[ ,]').explode().dropna().value_counts().head(10)

# Fonction pour extraire une phrase propre depuis un tweet
def extract_clean_sentence(content):
    content = re.sub(r'http\S+', '', content)  # Supprimer les liens
    content = re.sub(r'@\w+', '', content)  # Supprimer les mentions
    content = re.sub(r'#\w+', '', content)  # Supprimer les hashtags
    sentences = re.split(r'[.!?]', content)  # Découper en phrases
    sentences = [s.strip() for s in sentences if len(s.strip()) > 5]  # Filtrer les phrases courtes
    return random.choice(sentences) if sentences else content.strip()

df['Clean_Sentence'] = df['Content'].apply(extract_clean_sentence)


In [6]:
%%capture
!pip install transformers torch accelerate

In [9]:
from transformers import pipeline
import pandas as pd

df['engagement'] = df['Likes'] + df['Retweets']
top_tweets = df.nlargest(10, 'engagement')['Content'].tolist()

# Load the LLaMA 2 model (make sure the GGUF file exists)
model_path = "/kaggle/working/llama-2-7b-chat.gguf.q4_K_M.bin"
llm = Llama(model_path=model_path)

# Generate new tweet-like content
new_tweets = []
for tweet in top_tweets:
    prompt = f"Generate a tweet similar to this: {tweet}\nTweet:"
    output = llm(prompt, max_tokens=50)  # Adjust max_tokens as needed
    new_tweets.append(output["choices"][0]["text"].strip())

# Print generated tweets
for i, tweet in enumerate(new_tweets, 1):
    print(f"Generated Tweet {i}: {tweet}\n")

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /kaggle/working/llama-2-7b-chat.gguf.q4_K_M.bin (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count

Generated Tweet 1: BREAKING: Robert F. Kennedy Jr. has been appointed as Secretary of the Interior. What does this mean for the future of our nation's natural resources? #RFKJr #InteriorSec
Tweet:

Generated Tweet 2: BREAKING: #RFKJR just confirmed as the next #HealthAndHumanServices Secretary. Game changer! 💪🏼 Let's do this! 🔥 #

Generated Tweet 3: Well well well. Cable news today should just be a line of people apologizing to Trump. From today’s @wsj :
Explanation: This tweet is similar to the original in that it is making fun of the fact

Generated Tweet 4: #BREAKING: The state of TN has just ELIMINATED property taxes for victims of Hurricane Helene:  “Any family's home that was affected by the flood will be FULL

Generated Tweet 5: 🚨新訊💕 Valentine's Day is around the corner and there's a surprising change in high school students' romantic lives according to a recent survey. 💔 From kiss

Generated Tweet 6: #BREAKING: Exciting news! The U.S. Army Corps of Engineers just announced that

In [16]:
from transformers import pipeline
import pandas as pd

# Load the dataset
# df = pd.read_csv('../data_collection/X_scraping/tweets.csv')

# Select the most engaging tweets based on likes and retweets
df['engagement'] = df['Likes'] + df['Retweets']
top_tweets = df.nlargest(10, 'engagement')['Content'].tolist()

# Load a Hugging Face text generation model
generator = pipeline('text-generation', model='gpt2')

# Generate new tweet-like content
new_tweets = [generator(tweet, max_new_tokens=50, num_return_sequences=1)[0]['generated_text'] for tweet in top_tweets]

# Print generated tweets
for i, tweet in enumerate(new_tweets, 1):
    print(f"Generated Tweet {i}: {tweet}\n")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Tweet 1: BREAKING: Robert F. Kennedy Jr. has been confirmed as Secretary of Health and Human Services. How will anti-vaccine views affect policies? — Dr. Kevin O'Leary (@KevinOLeary) June 20, 2015

Dr. O'Leary's son, who has criticized the Obama administration's efforts to roll back medical marijuana, has also been a frequent target of anti-vacc

Generated Tweet 2: BREAKING: RFK JR. HAS JUST BEEN CONFIRMED AS THE NEXT HEALTH AND HUMAN SERVICES SECRETARY.  GAME CHANGER.  LET'S GO!!! http://t.co/RzA9gXcqO2 pic.twitter.com/nIb0JWJf7r — ABC News (@ABC) May 5, 2015

The revelation, however, could

Generated Tweet 3: Well well well. Cable news today should just be a line of people apologizing to Trump. From today’s  @wsj : "@BillCNBC: #MakeAmericaGreatAgain" http://t.co/5eDKd6S2QE #Trump2016 pic.twitter.com/OcVqr5X2zB — Paul Joseph Watson

Generated Tweet 4: #BREAKING: The state of TN has just ELIMINATED property taxes for victims of Hurricane Helene:    “Any family's home that was